In [1]:
import os

# = = = = = = = = = = = = = = =

import pandas as pd

# = = = = = = = = = = = = = = =

import requests

import sys
sys.path.append('../00.Tools')
from crawler_configuration import get_header, get_proxy

from bs4 import BeautifulSoup
from lxml import etree

# = = = = = = = = = = = = = = =

import json

In [2]:
directory_in = './file/1.url'

list_file = [file for file in sorted(os.listdir(directory_in)) if file not in ['.DS_Store'] and os.path.isfile(f'{directory_in}/{file}')]
file = list_file[0]

file

'crawler_1.xlsx'

In [3]:
list_dict_correct, list_series_error, df_input = [], [], pd.read_excel(f'{directory_in}/{file}',
                                                                       header=0,
                                                                       dtype=str).fillna('')

df_input['No'] = df_input['No'].astype(int)

count_remain = len(df_input)

print(f'总数量：{count_remain}')
print()
df_input

总数量：10



,No,Url
0,1,https://www.cardone.com/product/windshield-wip...
1,2,https://www.cardone.com/product/windshield-wip...
2,3,https://www.cardone.com/product/windshield-wip...
3,4,https://www.cardone.com/product/windshield-wip...
4,5,https://www.cardone.com/product/windshield-wip...
5,6,https://www.cardone.com/product/windshield-wip...
6,7,https://www.cardone.com/product/windshield-wip...
7,8,https://www.cardone.com/product/windshield-wip...
8,9,https://www.cardone.com/product/windshield-wip...
9,10,https://www.cardone.com/product/windshield-wip...


In [4]:
(crawler_index, crawler_series) = next(df_input.iterrows())

crawler_index, crawler_series

(0,
 No                                                     1
 Url    https://www.cardone.com/product/windshield-wip...
 Name: 0, dtype: object)

In [5]:
import time

count_retry = 0
while True:
    count_retry += 1
    print(count_retry)
    try:
        url_request = crawler_series['Url']

        resp = requests.get(url_request,
                            headers=get_header(),
                            proxies=get_proxy(),
                            timeout=(5, 15))

        if resp.status_code == 200:
            soup = BeautifulSoup(resp.text, 'lxml')
            html = etree.HTML(str(soup))

            title = html.xpath('//head/title/text()')[0].strip()
            with open('./part.html', 'w', encoding='UTF-8') as file:
                file.write(str(soup))
            break
    except KeyboardInterrupt:
        break
    except:
        time.sleep(3)
        continue

title

1


'Windshield Wiper Motor - 40-1000'

In [6]:
subtitle = html.xpath('//h2[@itemprop="brand"]/a/span[@itemprop="name"]/text()')[0].strip()

subtitle

'CARDONE Reman'

In [7]:
part_number = html.xpath('//dd[@itemprop="sku"]/text()')[0].strip()

part_number

'40-1000'

In [8]:
dict_src, list_src = {}, html.xpath('//li[contains(@class, "productView-thumbnail")]/a[@class="productView-thumbnail-link"]/@href')
for i, src in enumerate(list_src):
    dict_src[str(i)] = src.strip()

dict_src

{'0': 'https://cdn11.bigcommerce.com/s-z9vhe3o238/images/stencil/1280x1280/products/82816/5488626/482__00992.1687973871.jpg?c=1',
 '1': 'https://cdn11.bigcommerce.com/s-z9vhe3o238/images/stencil/1280x1280/products/82816/5488628/482__97909.1687973872.jpg?c=1',
 '2': 'https://cdn11.bigcommerce.com/s-z9vhe3o238/images/stencil/1280x1280/products/82816/5488630/482__09398.1687973874.jpg?c=1',
 '3': 'https://cdn11.bigcommerce.com/s-z9vhe3o238/images/stencil/1280x1280/products/82816/5488632/482__69563.1687973875.jpg?c=1',
 '4': 'https://cdn11.bigcommerce.com/s-z9vhe3o238/images/stencil/1280x1280/products/82816/5488635/482__93868.1687973876.jpg?c=1',
 '5': 'https://cdn11.bigcommerce.com/s-z9vhe3o238/images/stencil/1280x1280/products/82816/5488640/482__32832.1687973878.jpg?c=1',
 '6': 'https://cdn11.bigcommerce.com/s-z9vhe3o238/images/stencil/1280x1280/products/82816/5488643/482__89435.1687973879.jpg?c=1',
 '7': 'https://cdn11.bigcommerce.com/s-z9vhe3o238/images/stencil/1280x1280/products/82816/

In [9]:
type_, application_summary, list_dict, dict_count = '', '', json.loads(html.xpath('//div[@id="addon"]/@data-custom-fields')[0]), {}
for dict_ in list_dict:
    key = dict_['name'].strip()
    if key == 'Part Type':
        type_ = dict_['value'].strip()
    elif key == 'Application Summary':
        application_summary = dict_['value'].strip()
    else:
        if key in dict_count:
            dict_count[key] += 1
        else:
            dict_count[key] = 1

i, dict_info, dict_no = 0, {}, {}
for dict_ in list_dict:
    key = dict_['name'].strip()
    if key not in ['Part Type', 'Application Summary']:
        if dict_count[key] == 1:
            dict_info[str(i)] = {key: dict_['value'].strip()}
        else:
            if key in dict_no:
                dict_no[key] += 1
            else:
                dict_no[key] = 1
            dict_info[str(i)] = {f'''{key}-{dict_no[key]}''': dict_['value'].strip()}
        i += 1

type_, application_summary, dict_info

('Windshield Wiper Motor',
 'Chevrolet Camaro 1998-93; Pontiac Firebird 1998-93',
 {'0': {'productDeskExternalKey': 'CPCX|40-1000'},
  '1': {'Package Contents': 'Wiper Motor; Instruction Sheet'},
  '2': {'E-Waste': 'Yes'},
  '3': {'Product Condition': 'Remanufactured'},
  '4': {'Mounting Hole Quantity': '3'},
  '5': {'Terminal Type': 'Blade'},
  '6': {'Mounting Type': 'Eyelet; Grommet'},
  '7': {'Weight PG': '5.9000'},
  '8': {'Washer Pump Included': 'No'},
  '9': {'Warranty Info': '1 Year/18,000 Mile'},
  '10': {'Sustainable Choice': 'Yes'},
  '11': {'Lifecycle Status': 'Available to Order'},
  '12': {'ItemLevelGTIN': '00082617338332'},
  '13': {'_addonCardone': '40-1000::98271:100085:306'},
  '14': {'Eligible for R&R Service': 'Yes'}})

In [10]:
list_dict_correct.append({'No': crawler_series['No'],
                          'Type': type_,
                          'Part Number': part_number,
                          'Title': title,
                          'Application Summary': application_summary,
                          'Picture': '',
                          'Url': crawler_series['Url'],
                          'Subtitle': subtitle,
                          'Json_Src': json.dumps(dict_src),
                          'Json_Info': json.dumps(dict_info)})

list_dict_correct

[{'No': 1,
  'Type': 'Windshield Wiper Motor',
  'Part Number': '40-1000',
  'Title': 'Windshield Wiper Motor - 40-1000',
  'Application Summary': 'Chevrolet Camaro 1998-93; Pontiac Firebird 1998-93',
  'Picture': '',
  'Url': 'https://www.cardone.com/product/windshield-wiper-motor-40-1000',
  'Subtitle': 'CARDONE Reman',
  'Json_Src': '{"0": "https://cdn11.bigcommerce.com/s-z9vhe3o238/images/stencil/1280x1280/products/82816/5488626/482__00992.1687973871.jpg?c=1", "1": "https://cdn11.bigcommerce.com/s-z9vhe3o238/images/stencil/1280x1280/products/82816/5488628/482__97909.1687973872.jpg?c=1", "2": "https://cdn11.bigcommerce.com/s-z9vhe3o238/images/stencil/1280x1280/products/82816/5488630/482__09398.1687973874.jpg?c=1", "3": "https://cdn11.bigcommerce.com/s-z9vhe3o238/images/stencil/1280x1280/products/82816/5488632/482__69563.1687973875.jpg?c=1", "4": "https://cdn11.bigcommerce.com/s-z9vhe3o238/images/stencil/1280x1280/products/82816/5488635/482__93868.1687973876.jpg?c=1", "5": "https://c

In [11]:
crawler_status = True

crawler_status

True

In [12]:
crawler_series['Request_Url'] = url_request

list_series_error.append(crawler_series)

list_series_error

[No                                                             1
 Url            https://www.cardone.com/product/windshield-wip...
 Request_Url    https://www.cardone.com/product/windshield-wip...
 Name: 0, dtype: object]

In [13]:
if list_dict_correct:
    df_correct = pd.DataFrame(list_dict_correct).sort_values(by=['No'],
                                                             ascending=[True],
                                                             ignore_index=True)
    df_correct.to_excel('./test_part.xlsx', index=False)

df_correct

,No,Type,Part Number,Title,Application Summary,Picture,Url,Subtitle,Json_Src,Json_Info
0,1,Windshield Wiper Motor,40-1000,Windshield Wiper Motor - 40-1000,Chevrolet Camaro 1998-93; Pontiac Firebird 199...,,https://www.cardone.com/product/windshield-wip...,CARDONE Reman,"{""0"": ""https://cdn11.bigcommerce.com/s-z9vhe3o...","{""0"": {""productDeskExternalKey"": ""CPCX|40-1000..."


In [14]:
if list_series_error:
    df_error = pd.DataFrame(list_series_error).sort_values(by=['No'],
                                                           ascending=[True],
                                                           ignore_index=True)
    df_error.to_excel('./test_part_error.xlsx', index=False)

df_error

,No,Url,Request_Url
0,1,https://www.cardone.com/product/windshield-wip...,https://www.cardone.com/product/windshield-wip...
